# Advent of Code 2022

## Day 16: Proboscidea Volcanium

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

Today's notebook isn't too tidy. I didn't get finished until 8:00 PM.

For part 1 I read the graph of the caves, collapse all nodes with 0 flow rate down to shortest paths, ultimately build a distance matrix between the set containing all flow-rate > 0 nodes and the start node. I represent the solution as an ordered list of valves to open, e.g. for test input: `['DD', 'HH']` means go from AA to DD, open DD, go from DD to EE to FF to GG to HH. Intermediate nodes are implied. In evaluating, time jumps forward in steps of travel time plus 1 for opening. Venting score is remaining time times flow rate so I don't keep adding each time step.

I search all the possible sequences and in not too much time come up with the largest score.

Part 2 I enumerate al possible sequences one human/elephant can use to open the valves in 26 minutes. Then make all pairs of non-overlapping sequences. This takes my M1 Pro one hour to do! Finally in a shorter step I just loop over the pairs, adding up the score to find the highest.

Solution works. Did it within the day the puzzle went up, no spoilers, continued the 16-days streak. It's not a great solution but it works.

### Imports

In [ ]:
import random
from collections import defaultdict
from tqdm.notebook import tqdm as tqdm
from typing import Iterator, Optional
import re
from math import inf

### Reading Input

In [ ]:
def read_input(filename: str) -> Iterator[tuple[str, int, list[str]]]:
    with open(filename) as file:
        for line in file:
            line = line.strip()
            match = re.search(r'^Valve ([A-Z][A-Z]) has flow rate=([0-9][0-9]?); tunnel[a-z]* lead[a-z]* to valve[a-z]* ([A-Z, ]*)$', line)
            if match is None:
                raise ValueError(f're.search returned {match} on string "{line}"')
            if len(match.groups()) != 3:
                raise ValueError(f're.search returned {len(match.groups())} match grounps on string "{line}"')
            valve_number, flow_rate, leads_to = match.groups()
            yield valve_number, int(flow_rate), leads_to.split(', ')

In [ ]:
INPUT_FILE = 'data/input16.txt'
START = 'AA'

### Data

In [ ]:
key_nodes = {}
neighbours_raw = {}
for valve_number, flow_rate, leads_to in read_input(INPUT_FILE):
    if valve_number == START or flow_rate != 0:
        key_nodes[valve_number] = flow_rate
    neighbours_raw[valve_number] = leads_to

### Part 1

In [ ]:
def search(path, shortest):

    if path:

        # process path
        length = len(path) - 1
        head, tail = path[0], path[-1]
        assert (type(head) == str), f'{head = }, {path = }'
        assert (type(tail) == str), f'{tail = }, {path = }'
        best = shortest[(head, tail)]
        if length < best:
            shortest[(head, tail)] = length

            # all next neighbours
            for neighbour in neighbours_raw[tail]:
                search(path + [neighbour], shortest)

    else:

        # all start points
        for neighbour in neighbours_raw.keys():
            search([neighbour], shortest)

shortest = defaultdict(lambda: inf)
search([], shortest)

distance_matrix = {}
for x in sorted(key_nodes.keys()):
    distance_matrix[x] = {}
    for y in sorted(key_nodes.keys()):
        distance_matrix[x][y] = shortest[(x, y)]

In [ ]:
print('Nodes we can visit, with amount to vent:')
print(key_nodes)
print()

print('Shortest path distance between all pairs of nodes:')
for x in distance_matrix:
    print(x, distance_matrix[x])
print()

In [ ]:
# for a given sequence of vents, returns the total score
def score_sequence(sequence: list[str], total_time: int) -> Optional[int]:
    assert len(set(sequence)) == len(sequence)
    current_vertex = 'AA'
    time_remaining = total_time
    vented = 0
    for next_vertex in sequence:
        distance = distance_matrix[current_vertex][next_vertex]
        if time_remaining >= distance:
            current_vertex = next_vertex
            time_remaining -= distance
            if time_remaining == 1:
                return None
            elif time_remaining >= 0:
                time_remaining -= 1
                vented += key_nodes[current_vertex] * time_remaining
            else:
                return None
        else:
            return None
    return vented

In [ ]:
if 'test' in INPUT_FILE:
    example_sequence = ['DD', 'BB', 'JJ', 'HH', 'EE', 'CC']
    print(score_sequence([], 30))
    print(score_sequence(['DD'], 30))
    print(score_sequence(['DD', 'BB'], 30))
    print(score_sequence(['DD', 'BB', 'JJ'], 30))
    print(score_sequence(['DD', 'BB', 'JJ', 'HH'], 30))
    print(score_sequence(['DD', 'BB', 'JJ', 'HH', 'EE'], 30))
    print(score_sequence(['DD', 'BB', 'JJ', 'HH', 'EE', 'CC'], 30))
else:
    print('running on real file')

In [ ]:
BEST_SCORE = 0
BEST_SEQUENCE = []

def track_best(score, sequence):
    global BEST_SCORE
    global BEST_SEQUENCE
    if score > BEST_SCORE:
        BEST_SCORE = score
        BEST_SEQUENCE = sequence
        print(BEST_SCORE, BEST_SEQUENCE)


def search_problem(sequence: list[str], total_time: int, closed: set[str], callback):

    score = score_sequence(sequence, total_time=total_time)

    if score is None:
        return

    callback(score, sequence)

    for next_vent in closed:
        search_problem(sequence=sequence+[next_vent], total_time=total_time, closed=closed-{next_vent}, callback=callback)

search_problem(sequence=[], total_time=30, closed=set(key_nodes.keys())-{'AA'}, callback=track_best)
print()
print('DONE')
print()
print(f'Best Score : {BEST_SCORE}')
print(f'Best Sequence : {BEST_SEQUENCE}')

## Part 2

In [ ]:
ALL_VIABLE = {i: {} for i in range(len(key_nodes))} # no sub 1, want 0 to 15

def track_viable(score, sequence):
    global ALL_VIABLE
    ALL_VIABLE[len(sequence)][tuple(sequence)] = score


def search_problem(sequence: list[str], total_time: int, closed: set[str], callback):
    score = score_sequence(sequence, total_time=total_time)
    if score is None:
        return
    callback(score, sequence)
    for next_vent in closed:
        search_problem(sequence=sequence+[next_vent], total_time=total_time, closed=closed-{next_vent}, callback=callback)


search_problem(sequence=[], total_time=26, closed=set(key_nodes.keys())-{'AA'}, callback=track_viable)
print('DONE')

In [ ]:
MAXIMUM_VENTS = 0
for length, viable in ALL_VIABLE.items():
    print(length, len(viable))
    if len(viable) > 0:
        if MAXIMUM_VENTS < length:
            MAXIMUM_VENTS = length
print()
print(f'one person can open at most {MAXIMUM_VENTS} vents')
ALL_VIABLE = {k: v for k, v in ALL_VIABLE.items() if len(v) > 0}
ALL_VIABLE_UNGROUPED = {}
for length, viable in ALL_VIABLE.items():
    print(length, len(viable))
    for seq, score in viable.items():
        ALL_VIABLE_UNGROUPED[seq] = score
len(ALL_VIABLE_UNGROUPED)

In [ ]:
ALL_PAIRS = set()
X1 = list(ALL_VIABLE_UNGROUPED.keys())
random.shuffle(X1)
X2 = list(ALL_VIABLE_UNGROUPED.keys())
random.shuffle(X2)
for x in tqdm(X1, desc="creating all pairs of viable solves that don't overlap"):
    if len(x) > 0:
        for y in X2:
            if len(y) > 0:
                sx = set(x)
                sy = set(y)
                if len(sx | sy) == len(sx) + len(sy):
                    pair = [x, y]
                    pair.sort()
                    pair = tuple(pair)
                    ALL_PAIRS.add(pair)

In [ ]:
best_score = 0
for y, e in tqdm(ALL_PAIRS, desc='checking pairs'):
    score = ALL_VIABLE_UNGROUPED[y] + ALL_VIABLE_UNGROUPED[e]
    if score > best_score:
        best_score = score
        #print(score, y, e, sep='\t')

#print()
print('The answer is', best_score)